#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### CHAPTER 22
**CH22A How does a merger between airlines affect prices?**

 using the airline-tickets-usa dataset
 
 version 1.0 2021-05-05

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import sys
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

In [2]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/airline-tickets-usa/clean/"
data_out = dirname + "da_case_studies/ch22-airline-merger-prices/"
output = dirname + "da_case_studies/ch22-airline-merger-prices/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)

In [3]:
# Import the prewritten helper functions 
from py_helper_functions import *

## Data Preparation Steps and Descriptive Statistics

In [13]:
data = pd.read_stata(data_in+"originfinal-panel.dta")

In [14]:
data = data.loc[lambda x: (x["year"] == 2011) | (x["year"] == 2016)].reset_index(
    drop=True
)

In [15]:
# * create total number of passengers from shares
# * so we can get aggreagate shares

data["ptotalAA"] = data["shareAA"] * data["passengers"]
data["ptotalUS"] = data["shareUS"] * data["passengers"]
data["ptotallargest"] = data["sharelargest"] * data["passengers"]

data_agg = data.groupby(["origin", "finaldest", "return", "year"]).agg(
    airports=("airports", "first"),
    return_sym=("return_sym", "first"),
    stops=("stops", "first"),
    ptotalAA=("ptotalAA", sum),
    ptotalUS=("ptotalUS", sum),
    ptotallargest=("ptotallargest", sum),
    passengers=("passengers", sum),
    itinfare=("itinfare", sum),
).reset_index()

In [16]:
data_agg["after"] = np.where(data_agg["year"] == 2016, 1, 0)
data_agg["before"] = np.where(data_agg["year"] == 2011, 1, 0)
data_agg["avgprice"] = data_agg["itinfare"] / data_agg["passengers"]
data_agg["shareAA"] = data_agg["ptotalAA"] / data_agg["passengers"]
data_agg["shareUS"] = data_agg["ptotalUS"] / data_agg["passengers"]
data_agg["sharelargest"] = data_agg["ptotallargest"] / data_agg["passengers"]
data_agg["AA"] = np.where(data_agg["shareAA"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["US"] = np.where(data_agg["shareUS"] > 0, 1, 0)
data_agg["AA_and_US"] = np.where(
    (data_agg["shareAA"] > 0) & (data_agg["shareUS"]) > 0, 1, 0
)
data_agg["AA_or_US"] = np.where(
    (data_agg["shareAA"] > 0) | (data_agg["shareUS"]) > 0, 1, 0
)

In [17]:
# create numeric ID for market
data_agg["market"] = (
    data_agg["origin"]
    + "_"
    + data_agg["finaldest"]
    + "_"
    + data_agg["return"].apply(str)
)

In [18]:
# passengers before and after
data_agg["pass_bef_id"] = np.where(data_agg["before"] == 1, data_agg["passengers"], np.nan)
data_agg["pass_aft_id"] = np.where(data_agg["after"] == 1, data_agg["passengers"], np.nan)

data_agg["pass_bef"] = data_agg.groupby("market")["pass_bef_id"].transform(np.nanmean)
data_agg["pass_aft"] = data_agg.groupby("market")["pass_aft_id"].transform(np.nanmean)

In [19]:
# balanced vs unbalanced part of panel
data_agg["balanced"] = data_agg.groupby("market")["origin"].transform("count") == 2

In [20]:
data_agg.groupby("balanced").agg(
    sum_passengers=("passengers", sum), n=("origin", "count")
)

,sum_passengers,n
balanced,,
False,561060.0,51516
True,54136421.0,226094


In [21]:
# Define treated and untreated markets
# treated: both AA and US present in the before period
# untreated: neither AA nor US present in the before period
# drop if only AA or only US in before period (neither treated nor untreated)

In [35]:
data_agg = data_agg.merge(
    data_agg.loc[lambda x: x["balanced"] == 1]
    .assign(
        treated=lambda x: (x["AA_and_US"] == 1) & (x["before"] == 1),
        untreated=lambda x: (x["AA_and_US"] == 0) & (x["before"] == 1),
        smallmkt=lambda x: (x["passengers"] < 5000) & (x["before"] == 1),
    )
    .groupby("market")
    .agg(
        treated=("treated", max),
        untreated=("untreated", max),
        smallmkt=("smallmkt", max),
    )
    .reset_index(),
    on="market",
    how="left",
)

In [36]:
data_agg["lnavgp"] = np.where(np.isinf(np.log(data_agg["avgprice"])), np.nan, np.log(data_agg["avgprice"]))
data_agg["d_lnavgp"] = data_agg.groupby("market")["lnavgp"].transform("diff")

## Describe

In [37]:
#describe yearly data
data_agg[["year","passengers"]].groupby("year").describe(percentiles=[.25, .5, .75,.9]).round(0)

passengers                                                    
          count   mean     std  min  25%  50%   75%    90%      max
year                                                               
2011   141712.0  170.0   980.0  1.0  2.0  7.0  37.0  205.0  39853.0
2016   135898.0  225.0  1270.0  1.0  2.0  8.0  45.0  245.0  46881.0

In [38]:
data_agg.query('(origin=="JFK") & (finaldest=="LAX")')[
    ["market", "origin", "finaldest", "return", "year", "passengers"]
]

,market,origin,finaldest,return,year,passengers
139047,JFK_LAX_0,JFK,LAX,0,2011,31312.0
139048,JFK_LAX_0,JFK,LAX,0,2016,46881.0
139049,JFK_LAX_1,JFK,LAX,1,2011,37861.0
139050,JFK_LAX_1,JFK,LAX,1,2016,41927.0


In [39]:
data_agg.query("year == 2011")[["smallmkt", "passengers"]].groupby("smallmkt").describe().round(0)

passengers                                                          
              count    mean     std     min     25%     50%      75%      max
smallmkt                                                                     
False         887.0  9813.0  5748.0  5003.0  5987.0  7908.0  11278.0  39853.0
True       112160.0   136.0   456.0     1.0     3.0    12.0     55.0   4994.0

In [40]:
# describe balanced
data_agg.groupby(["year", "balanced"])["passengers"].describe(percentiles=[]).round(1)

count   mean     std  min   50%      max
year balanced                                             
2011 False      28665.0    3.6    23.8  1.0   1.0   2051.0
     True      113047.0  211.9  1092.7  1.0  12.0  39853.0
2016 False      22851.0   20.0   178.1  1.0   1.0   4798.0
     True      113047.0  267.0  1386.3  1.0  13.0  46881.0

In [41]:
# describe treatment
data_agg.groupby(["year", "treated", "untreated"]).describe(percentiles=[]).round(1)

return                           airports            \
                           count mean  std  min  50%  max     count mean  std   
year treated untreated                                                          
2011 False   True       101418.0  0.5  0.5  0.0  1.0  1.0  101418.0  4.5  1.4   
     True    False       11629.0  0.5  0.5  0.0  0.0  1.0   11629.0  4.1  1.2   
2016 False   True       101418.0  0.5  0.5  0.0  1.0  1.0  101418.0  4.4  1.4   
     True    False       11629.0  0.5  0.5  0.0  0.0  1.0   11629.0  4.1  1.2   

                             ... lnavgp                 d_lnavgp            \
                        min  ...    std  min  50%  max     count mean  std   
year treated untreated       ...                                             
2011 False   True       2.0  ...    0.7  0.0  6.0  8.5       0.0  NaN  NaN   
     True    False      2.0  ...    0.6  2.2  5.7  7.5       0.0  NaN  NaN   
2016 False   True       2.0  ...    0.7  0.4  6.1  8.9  101004.0  0.1  0.7   
     True    False      2.0  ...    0.7  1.6  5.8  7.8   11628.0  0.0  0.3   

                                       
                        min  50%  max  
year treated untreated                 
2011 False   True       NaN  NaN  NaN  
     True    False      NaN  NaN  NaN  
2016 False   True      -6.4  0.1  6.5  
     True    False     -2.4  0.0  2.3  

[4 rows x 150 columns]

In [42]:
# describe outcome
data_agg.query("before == 1")["avgprice"].describe().round(2)

count    141712.00
mean        455.75
std         276.49
min           0.00
25%         285.50
50%         413.40
75%         561.03
max        6054.00
Name: avgprice, dtype: float64

In [43]:
data_agg.query("avgprice == 0")["passengers"].describe().round(2)

count    843.00
mean       1.08
std        0.31
min        1.00
25%        1.00
50%        1.00
75%        1.00
max        3.00
Name: passengers, dtype: float64

In [44]:
data_agg.to_pickle(data_out + "ch22-airline-workfile.pkl")